In [1]:
from complex_exponential import *
from scipy.optimize import minimize

In [2]:

def f(x):
    theseparams=x.reshape(3,-1)*mp.mpf(1)

    for i in range(theseparams.shape[1]):
        a,b,c=theseparams[:,i].flatten()
        if a<0 or b<0 or c<0 or (a==0 and b==0) or (a==0 and c==0) or (b==0 and c==0):
            return 1000
        
    #symetry breaking condition reduces space of solutions synce its symetric
    for i in range(theseparams.shape[1]-1):
        if mp.norm(theseparams[:,i])>mp.norm(theseparams[:,i+1]):
            return 1000
        
    

    subspace=Subspace(theseparams.shape[1])

    subspace.set_N_func(N_func)
    subspace.set_H_func(H_func)

    subspace.set_params(theseparams)

    subspace.make_N_mat()
    subspace.make_H_mat()
    subspace.find_N_eigens()
    subspace.make_Y_mat()
    subspace.make_invs_sqrt_beta_mats()
    subspace.make_P_mats()
    subspace.find_P_eigens()
    subspace.find_energy_levels()
    return np.float64(subspace.energy_levels[0])

res=minimize(f,x0=[400,500,600,700,1,2,3,4,0.02,0.03,0.04,0.05],method="Nelder-Mead")
print(res)

       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: -402.63730067250907
             x: [ 4.021e+02  5.169e+02  5.427e+02  7.403e+02  9.774e-01
                  1.995e+00  3.340e+00  3.876e+00  2.251e-02  3.274e-02
                  4.187e-02  2.909e-02]
           nit: 566
          nfev: 1016
 final_simplex: (array([[ 4.021e+02,  5.169e+02, ...,  4.187e-02,
                         2.909e-02],
                       [ 4.021e+02,  5.169e+02, ...,  4.187e-02,
                         2.909e-02],
                       ...,
                       [ 4.021e+02,  5.169e+02, ...,  4.187e-02,
                         2.909e-02],
                       [ 4.021e+02,  5.169e+02, ...,  4.187e-02,
                         2.909e-02]]), array([-4.026e+02, -4.026e+02, ..., -4.026e+02, -4.026e+02]))


In [4]:

def f(x):
    theseparams=x[:6].reshape(3,-1)*mp.mpf(1)+x[6:].reshape(3,-1)*mp.mpf(1)*1j

    if np.any(x[:6]<=0):
        return 1000

    #symetry breaking condition reduces space of solutions synce its symetric
    for i in range(theseparams.shape[1]-1):
        if mp.norm(theseparams[:,i])>mp.norm(theseparams[:,i+1]):
            return 1000
        
    
    subspace=Subspace(theseparams.shape[1])
    print(theseparams)

    subspace.set_N_func(N_func)
    subspace.set_H_func(H_func)

    subspace.set_params(theseparams)

    subspace.make_N_mat()

    subspace.make_H_mat()

    print(subspace.H_mat)
    subspace.find_N_eigens()
    subspace.make_Y_mat()
    subspace.make_invs_sqrt_beta_mats()
    subspace.make_P_mats()
    subspace.find_P_eigens()
    subspace.find_energy_levels()
    return np.float64(subspace.energy_levels[0])

In [10]:
f(np.array([400,500,1,2,0.02,0.03,0,0.01,0,0.01,0,0.01]))

[[mpc(real='400.0', imag='0.0')
  mpc(real='500.0', imag='0.01000000000000000020816681711721685132943093776702880859375')]
 [mpc(real='1.0', imag='0.0')
  mpc(real='2.0', imag='0.01000000000000000020816681711721685132943093776702880859375')]
 [mpc(real='0.0200000000000000004163336342344337026588618755340576171875', imag='0.0')
  mpc(real='0.0299999999999999988897769753748434595763683319091796875', imag='0.01000000000000000020816681711721685132943093776702880859375')]]
[                                                                                                  (-402.6257393165248774290481809482846388063632715950250924026061204213477661012473352262323395970124011 + 0.0j)  (-332.4330091783853204962015987582987109370246414781718724519041723370061372947755307196566717858384443 + 1.628168699397616096171836866805321246407971492753674713178207781204791479349298115683862764244794264j)]
[(-332.2390189509040195987579806142570758698187673693125232412015078455474048666407145357607241497901933

np.float64(-409.9849471039512)

In [11]:
H_func(0,0,[mp.mpc(400)],[mp.mpc(1)],[mp.mpc(0.02j)])

mpc(real='-402.6257385432368878224330045157848061969899466071619344012737094904004108391700865261142979378897836185', imag='1.089409857315918342424418694478528101977601713815479973970666819326337356098609842753161395835476465759e-100')

In [9]:
res=minimize(f,x0=[400,500,1,2,0.02,0.03,0,0.01,0,0.01,0,0.01],method="Nelder-Mead")
print(res)

[[mpc(real='400.0', imag='0.0')
  mpc(real='500.0', imag='0.01000000000000000020816681711721685132943093776702880859375')]
 [mpc(real='1.0', imag='0.0')
  mpc(real='2.0', imag='0.01000000000000000020816681711721685132943093776702880859375')]
 [mpc(real='0.0200000000000000004163336342344337026588618755340576171875', imag='0.0')
  mpc(real='0.0299999999999999988897769753748434595763683319091796875', imag='0.01000000000000000020816681711721685132943093776702880859375')]]
[                                                                                                  (-402.6257393165248774290481809482846388063632715950250924026061204213477661012473352262323395970124011 + 0.0j)  (-332.4330091783853204962015987582987109370246414781718724519041723370061372947755307196566717858384443 + 1.628168699397616096171836866805321246407971492753674713178207781204791479349298115683862764244794264j)]
[(-332.2390189509040195987579806142570758698187673693125232412015078455474048666407145357607241497901933

KeyboardInterrupt: 